<a href="https://colab.research.google.com/github/janimiyarj/HuggingFace-Practise/blob/main/Hugging_Face_Practise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 552.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [5]:
# Import necessary libraries
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

# Define a function to load the model and tokenizer
def load_model_and_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        return None, None

# Load the Hugging Face model
model_name = "facebook/blenderbot-400M-distill"  # or any other suitable model
model, tokenizer = load_model_and_tokenizer(model_name)

# Define the prompt template
prompt_template_text = """You are a high school history teacher grading homework assignments. \
Based on the homework question indicated by “**Q:**” and the correct answer indicated by “**A:**”, your task is to determine whether the student's answer is correct. \
Grading is binary; therefore, student answers can be correct or wrong. \
Simple misspellings are okay.

**Q:** {question}
**A:** {correct_answer}

**Student's Answer:** {student_answer}
"""

prompt = PromptTemplate(input_variables=["question", "correct_answer", "student_answer"], template=prompt_template_text)

class GradingOutputParser(BaseOutputParser):
    def parse(self, text: str, correct_answer: str) -> str:
        # Convert both the response and correct answer to lowercase for case-insensitive comparison
        text_lower = text.lower()
        correct_answer_lower = correct_answer.lower()

        # Split the correct answer into keywords (can be improved further)
        correct_keywords = correct_answer_lower.split()

        # Check if the response contains the key terms from the correct answer
        if all(keyword in text_lower for keyword in correct_keywords):
            return "Correct"
        else:
            return "Wrong"

# Instantiate the parser
grading_parser = GradingOutputParser()

# Define a function to generate the response using the Hugging Face model
def generate_response(question, correct_answer, student_answer):
    try:
        # Prepare the input
        input_text = prompt.format(question=question, correct_answer=correct_answer, student_answer=student_answer)
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

        # Generate the response
        output = model.generate(**inputs)
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Print the raw model response to understand it better
        print("Raw Model Response:", response)

        grading_result = grading_parser.parse(response, correct_answer)

        return grading_result
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

# Example usage
question = "Who was the first president of the United States?"
correct_answer = "George Washington"
student_answer = "George Wasington"

response = generate_response(question, correct_answer, student_answer)
if response:
    print("Model's Grading:", response)


Raw Model Response:  George W. Washington was a US senior citizen who served in the American Revolution.
Model's Grading: Correct
